In [1]:
## Important libraries
import streamlit as st
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import chromadb
import uuid
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
load_dotenv()

True

In [3]:
# pip install langgraph
# pip install sentence-transformers

In [4]:
# pip install lxml

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.geeksforgeeks.org/ml-machine-learning/")
page_data = loader.load()

In [12]:
print(page_data)

[Document(metadata={'source': 'https://www.geeksforgeeks.org/ml-machine-learning/', 'title': 'What is Machine Learning? - GeeksforGeeks', 'description': 'Machine learning is a crucial subset of artificial intelligence that enables algorithms to learn from data, make predictions, and improve performance across various applications, including finance, healthcare, and autonomous systems, without explicit programming.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhat is Machine Learning? - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\nCoursesGet IBM CertificationsComplete Machine Learning & Data Science ProgramData Science Training ProgramData Analytics Training using Excel, SQL, Python & PowerBIComplete Data Analytics ProgramDSA to DevelopmentFor Working ProfessionalsData Structure & Algorithm Classes (Live)System 

In [8]:
# Model for generating Embedding
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

C:\Users\dheeraj\AppData\Local\Temp\ipykernel_18484\2773592769.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [8]:
llm=ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="deepseek-r1-distill-llama-70b"
)

In [9]:
response=llm.invoke("who are you")
response

AIMessage(content="<think>\n\n</think>\n\nGreetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 6, 'total_tokens': 50, 'completion_time': 0.16, 'prompt_time': 0.003526382, 'queue_time': 0.486874766, 'total_time': 0.163526382}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_3de5b607fa', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d675a4d-61a4-4b71-a3f3-1d897038c1e0-0', usage_metadata={'input_tokens': 6, 'output_tokens': 44, 'total_tokens': 50})

## Chatbot without memory with llama3.2 embedding

In [10]:
'''
# Load the vector store
vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)

query = "what is the symptom of Ebola"
# results = vectorstore.similarity_search(query, k=3) 
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2}) 
results = retriever.get_relevant_documents(query)

for doc in results:
    print(doc.page_content)


response=llm.invoke("what is symptom of Ebola")
print(response)

context = " ".join([doc.page_content for doc in results])
prompt = f"Based on the following information, answer the question:\n\n{context}\n\nQuestion: {query}"
response = llm.invoke(prompt)
'''

'\n# Load the vector store\nvectorstore = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)\n\nquery = "what is the symptom of Ebola"\n# results = vectorstore.similarity_search(query, k=3) \nretriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2}) \nresults = retriever.get_relevant_documents(query)\n\nfor doc in results:\n    print(doc.page_content)\n\n\nresponse=llm.invoke("what is symptom of Ebola")\nprint(response)\n\ncontext = " ".join([doc.page_content for doc in results])\nprompt = f"Based on the following information, answer the question:\n\n{context}\n\nQuestion: {query}"\nresponse = llm.invoke(prompt)\n'

## Chatbot with BAAI/bge-small-en-v1.5 model

In [11]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=200)
documents=text_splitter.split_documents(page_data)
len(documents)

21

In [12]:
# Create Chroma vector database
vectorstore = Chroma.from_documents(documents, embedding_model, persist_directory="chroma-BAAI")

# Save the database
vectorstore.persist()

C:\Users\dheeraj\AppData\Local\Temp\ipykernel_11644\1870202547.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [16]:
# Load the vector store
vectorstore = Chroma(persist_directory="chroma-BAAI", embedding_function=embedding_model)
query = "what is ML"
results = vectorstore.similarity_search(query, k=1) 
# print(results)
for doc in results:
    print(doc.page_content)

topic.Learning ObjectivesUnderstand the concept of Large Language Models (LLMs) and their importance in natural language processing.Know about different types of popular LLMs, such as BERT, GPT-3, and T5.Discuss the applications and use cases of Open Source LLMs.Hugging Face APIs for LLMs.Explore the future implications of LLMs, including their potential impact on job markets, communication, and society as a whole.What is LLM?A large language model (LLM) is a type of artificial intelligence (AI) that can generate human-like text and perform various natural language processing tasks. LLMs are trained on massive amounts of text data, which allows them to learn the patterns and relationships that exist in language. This makes them capable of generating text that is often indistinguishable


In [17]:
context = " ".join([doc.page_content for doc in results])
prompt = f"Based on the following information, answer the question:\n\n{context}\n\nQuestion: {query}"
response = llm.invoke(prompt)
print(response.content)

<think>
Okay, so I need to figure out what ML stands for in the context of the provided information. The question is asking, "what is ML?" Let me start by looking through the given text to see if ML is mentioned anywhere.

Looking at the topic section, the learning objectives mention Large Language Models (LLMs) and their importance in natural language processing. It talks about different types like BERT, GPT-3, and T5. It also discusses applications of Open Source LLMs, Hugging Face APIs, and future implications of LLMs. The definition provided is about LLMs, explaining they're a type of AI that can generate human-like text and perform NLP tasks.

Wait, the question is about ML, not LLM. Hmm, ML isn't directly mentioned in the provided text. But I know from general knowledge that ML stands for Machine Learning. Since the topic is about LLMs, which are a part of AI, and AI is closely related to ML, maybe the user is asking about ML in this context.

So, putting it together, ML stands f

In [36]:
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    memory=memory,
)

C:\Users\dheeraj\AppData\Local\Temp\ipykernel_11432\1824668749.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [7]:
chat_history = []
while True:
    query = input("Enter your question (or type 'exit' to quit): ")
    
    if query.lower() == "exit":
        break
    
    # Invoke the QA chain
    response = qa_chain.invoke({"question": query, "chat_history": chat_history})
    
    # Extract and clean response
    cleaned_response = re.sub(r"<think>.*?</think>", "", response["answer"], flags=re.DOTALL)
    

    # Generate follow-up questions
    follow_up_prompt = f"Generate 2-3 follow-up questions related to: {query}"
    follow_up_response = llm.invoke(follow_up_prompt)
    follow_up_questions = follow_up_response.content.split("\n")

    # Update chat history
    chat_history.append(HumanMessage(query))
    chat_history.append(AIMessage(cleaned_response))

    # Display response
    print("\n🤖 Bot:", cleaned_response)
    print("\n💡 Follow-up Questions:")
    for q in follow_up_questions:
        print("-", q.strip())

### Code for memory based Chatbot

In [25]:
tools = []
memory = MemorySaver()
tool_node = ToolNode(tools)
model = llm
bound_model = model.bind_tools(tools)
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    # return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
def should_continue(state: MessagesState):
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    return "action"
# Define the function that calls the model
def call_model(state: MessagesState):
    response = bound_model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next, we pass in the path map - all the possible nodes this edge could go to
    ["action", END],
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

In [26]:
config = {"configurable": {"thread_id": "5"}}

In [27]:
memory = MemorySaver()
query="What is LLM"

input_message = HumanMessage(content=f"{query}")
for event in app.stream({"messages": [query]}, config, stream_mode="values"):
    cleaned_response = re.sub(r"<think>.*?</think>", "", event["messages"][-1].content, flags=re.DOTALL)
    print(cleaned_response)
    # event["messages"][-1].pretty_print()

What is LLM


LLM stands for **Large Language Model**. It refers to a type of artificial intelligence (AI) system designed to process and understand human language at a large scale. These models are trained on vast amounts of text data, enabling them to generate coherent and contextually relevant text, answer questions, summarize content, and perform various language-related tasks.

### Key Features of LLMs:
1. **Training Data**: LLMs are trained on massive datasets, often sourced from the internet, books, or other text-based resources.
2. **Neural Networks**: They are built using deep learning techniques, typically based on transformer architectures, which are well-suited for handling sequential data like text.
3. **Generative Capabilities**: LLMs can generate text, making them useful for tasks like writing, translation, and creative content creation.
4. **Contextual Understanding**: They can understand context and maintain coherence in longer pieces of text, unlike simpler language m

In [ ]:
# pip install -qU langchain-community faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=200)
documents=text_splitter.split_documents(page_data)

In [3]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
vector_store.add_documents(documents=documents)

In [16]:
vectorstore = FAISS.from_documents(documents, embedding_model)

In [9]:
vectorstore = Chroma.from_documents(documents, embedding_model)
results = vectorstore.similarity_search("what is LLM", k=1)
results

[Document(metadata={'description': 'Large language models (LLMs) are powerful machine-learning models that can understand and generate natural language. They are trained on massive datasets of text and code, which allows them to learn…', 'language': 'en', 'source': 'https://medium.com/@akash.kesrwani99/what-arellm-large-language-model-51d1315acaf4', 'title': 'What are LLM(Large Language Model)? | by Akash Kesrwani | Medium'}, page_content='What are LLM(Large Language Model)? | by Akash Kesrwani | MediumOpen in appSign upSign inWriteSign upSign inWhat are LLM(Large Language Model)?Akash Kesrwani·Follow9 min read·Jul 3, 2023--2ListenShareLarge language models (LLMs) are powerful machine-learning models that can understand and generate natural language. They are trained on massive datasets of text and code, which allows them to learn the patterns and relationships that exist in language. LLMs can be used for a variety of tasks, such as translating languages, analyzing sentiment, and gener